<a href="https://colab.research.google.com/github/Loic6834568/Cosc_3P99/blob/main/Cosc_3P99_Loic_wedji_6834568_Notebook_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Intro for GitHub

Suppose we are working at Netflix. one of the system main function is recommend "relevant" movies to users.
-- recommender system
-- user rate movies . this can be modelled as a graph. Edges link nodes of two types (a user and a movie).

The concept of building machine learning models over data stored in a graph is what define Graph Neural Network. But why do we need it? Well, GNN's are very usefull in many aspects, for example transportation networks or social networks.It can be used to create predition or acurate guess based on a data it was given. In this project, we would focus on a machine learning model that predicts which movies a user might enjoy based on it's reviews.


##Libraries
The first approach is to make sure we have the correct libraries and packages installed.

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install torch_geometric
#!pip install torch_scatter torch_sparse torch_cluster

2.4.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
from torch_geometric.datasets import MovieLens
dataset = MovieLens(root="",model_name="all-MiniLM-L6-v2")
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')




Extracting ./raw/ml-latest-small.zip
Processing...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

Dataset: MovieLens():
Number of graphs: 1
Number of features: {'movie': 404, 'user': 0}


Done!


In [ ]:
data = dataset[0]
data

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836],
    time=[100836],
  }
)

In [ ]:
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
#print(f'Number of training nodes: {data.train_mask.sum()}')
#print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Number of nodes: 10352
Number of edges: 100836
Average node degree: 9.74
Has isolated nodes: True
Has self-loops: False
Is undirected: False


#Task for notebook


Notebook 2 - Two challenges. 1-Nodes have type 2 - From  node classificaiton to Link Prediction (-> we use node embedding to get the node embedding to create edge embeddings)

Link Prediciton -> New loss (MSE Loss)

https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py
pytorch_geometric/examples/hetero/hetero_link_pred.py at master · pyg-team/pytorch_geometric
Graph Neural Network Library for PyTorch. Contribute to pyg-team/pytorch_geometric development by creating an account on GitHub.


In [ ]:
from torch_geometric.nn.models import LightGCN

model = LightGCN(num_nodes=data.num_nodes, embedding_dim=2, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Define optimizer.

def train():
    model.train()
    optimizer.zero_grad()
    #pred = model(train_data.x_dict, train_data.edge_index_dict,train_data['user', 'movie'].edge_label_index)
    #loss = model.loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test(data):
    model.eval()
    #pred = model(data.x_dict, data.edge_index_dict,data['user', 'movie'].edge_label_index)
    #rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)


for epoch in range(1, 301):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

NameError: name 'train_data' is not defined